In [289]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

### Eerst gaan we alle datafiles die we nodig hebben mergen tot 1 dataframe

#### 1. Alles van Account mergen

##### Account mergen met activiteit tabellen

In [290]:
data_acc = pd.read_csv('../data_clean/Account_fixed.csv', sep=',')
data_acti = pd.read_csv('../data_clean/Activiteitscode_fixed.csv', sep=",")
data_acc_acti = pd.read_csv('../data_clean/Account_activiteitscode_fixed.csv', sep=",")

data_acc = data_acc[data_acc['account_reden_van_status'] != 'Inactief']
data_acc = data_acc[data_acc['account_reden_van_status'] != 'Stopzetting']
data_acc = data_acc[data_acc['account_adres_provincie'] == 'Oost-Vlaanderen']
data_acti = data_acti[data_acti['activiteitscode_status'] == 'Actief']

# merge account_activiteitscode with activiteitscode
merged_relation = data_acc_acti.merge(data_acti, left_on='account_activiteitscode_activiteitscode', right_on='activiteitscode_activiteitscode_id', how='inner')
# merge account with the merged relation from above
accounts_merged = data_acc.merge(merged_relation, left_on='account_account_id', right_on='account_activiteitscode_account', how='left').fillna('Geen')
accounts_merged.shape

(41934, 22)

In [291]:
# RAM vrijmaken
try:
    del data_acc
    del data_acti
    del data_acc_acti
    del merged_relation
except:
    print("Data is niet meer beschikbaar / is al verwijderd")

In [292]:
accounts_merged.shape

(41934, 22)

##### Accounts mergen met financiële data

In [293]:
# financiele_data = pd.read_csv('../data_clean/Account_financiële_data_fixed.csv', sep=',')
# financiele_data.shape

In [294]:
# accounts_merged = accounts_merged.merge(financiele_data, left_on='account_account_id', right_on='financieledata_ondernemingid', how='inner')
# accounts_merged.shape

##### Accounts merge met Contactfiches

In [295]:
contactfiches = pd.read_csv('../data_clean/Contact_fixed.csv', sep=',')
contactfiches = contactfiches[contactfiches['contact_status'] != 'Inactief']
contactfiches['contact_functietitel'] = contactfiches['contact_functietitel'].str.lower().str.replace(r'[^\w\s]', '', regex=True) \
                                                        .str.replace('  ', ' ').str.strip()
contactfiches.shape

(393514, 6)

In [296]:
accounts_merged = accounts_merged.merge(contactfiches, left_on='account_account_id', right_on='contact_account', how='inner')
accounts_merged.shape

(67949, 28)

##### Accounts mergen met Functie

In [297]:
contactfiches_functies = pd.read_csv('../data_clean/Contact_functie_fixed.csv', sep=',')
functies = pd.read_csv('../data_clean/Functie_fixed.csv', sep=',')
contactfiches_functies.shape, functies.shape

((589129, 2), (71, 2))

In [298]:
contactfiches_functies.columns

Index(['contactfunctie_contactpersoon', 'contactfunctie_functie'], dtype='object')

In [299]:
accounts_merged = accounts_merged.merge(contactfiches_functies, left_on='contact_contactpersoon_id', right_on='contactfunctie_contactpersoon', how='inner')

In [300]:
accounts_merged = accounts_merged.merge(functies, left_on='contactfunctie_functie', right_on='functie_functie_id', how='inner')
accounts_merged.shape

(91783, 32)

##### Accounts merge met Personen

In [301]:
df_persoon = pd.read_csv('../data_clean/Persoon_fixed.csv')

df_persoon = df_persoon[df_persoon['persoon_reden_van_status'] == 'Actief']

for col in df_persoon.columns:
    if col.__contains__('persoon_mail_regio'):
        df_persoon.drop(col, axis=1, inplace=True)

##############################################################################################################
#### ACCOUNT CONTACT PERSOON MERGE ####
##############################################################################################################

accounts_merged = accounts_merged.merge(df_persoon, left_on='contact_persoon_id', right_on='persoon_persoon_id', how='left')

# NaN waarden opvullen
for col in accounts_merged.columns:
    if col.__contains__('persoon_mail_type') or col.__contains__('persoon_mail_thema'):
        accounts_merged[col] = accounts_merged[col].fillna(0)

accounts_merged['persoon_marketingcommunicatie'] = accounts_merged['persoon_marketingcommunicatie'].fillna('-1')
accounts_merged['persoon_marketingcommunicatie'] = accounts_merged['persoon_marketingcommunicatie'] \
                                                            .str.replace('Strikt', '0').str.replace('Flexibel', '1') \
                                                            .str.replace('Uitgeschreven', '-1').str.replace('unknown', '-1').astype(int)

marketing_pressure_cols = [col for col in accounts_merged.columns if col.__contains__('persoon_mail_type') 
                           or col.__contains__('persoon_mail_thema') or col.__contains__('persoon_marketingcommunicatie')]

marketing_pressure_cols = [col for col in accounts_merged.columns if col.__contains__('persoon_mail_type') 
                           or col.__contains__('persoon_mail_thema') or col.__contains__('persoon_marketingcommunicatie')]

accounts_merged['marketing_pressure'] = accounts_merged[marketing_pressure_cols].sum(axis=1)
accounts_merged['marketing_pressure'] = accounts_merged['marketing_pressure'].astype(int)
accounts_merged.drop(marketing_pressure_cols, axis=1, inplace=True)

#### 2. Alles van Afspraken mergen

In [302]:
# RAM vrijmaken
try:
    # del financiele_data
    del contactfiches
    del contactfiches_functies
    del functies
except:
    print("Data is niet meer beschikbaar / is al verwijderd")

In [303]:
afspraak_betreft_account_cleaned = pd.read_csv('../data_clean/Afspraak_betreft_account_cleaned_fixed.csv', sep=',')
afspraak_account_gelinkt_cleaned = pd.read_csv('../data_clean/Afspraak_account_gelinkt_cleaned_fixed.csv', sep=',')
afspraak_betreft_contact_cleaned = pd.read_csv('../data_clean/Afspraak_betreft_contact_cleaned_fixed.csv', sep=',')

In [304]:
afspraak_betreft_account_cleaned.shape, afspraak_account_gelinkt_cleaned.shape, afspraak_betreft_contact_cleaned.shape

((4876, 7), (2934, 7), (2552, 7))

In [305]:
afspraak_betreft_account_cleaned = afspraak_betreft_account_cleaned.drop(['afspraak_betreft_account_betreft_id'], axis=1)
afspraak_account_gelinkt_cleaned = afspraak_account_gelinkt_cleaned.drop(['afspraak_account_gelinkt_account'], axis=1)
afspraak_betreft_contact_cleaned = afspraak_betreft_contact_cleaned.drop(['afspraak_betreft_contactfiche_betreft_id'], axis=1)

In [306]:
afspraak_betreft_account_cleaned.columns = ['afspraak_afspraak_id', 'afspraak_thema', 'afspraak_subthema', 'afspraak_onderwerp', 'afspraak_eindtijd', 'afspraak_keyphrases']
afspraak_account_gelinkt_cleaned.columns = ['afspraak_afspraak_id', 'afspraak_thema', 'afspraak_subthema', 'afspraak_onderwerp', 'afspraak_eindtijd', 'afspraak_keyphrases']
afspraak_betreft_contact_cleaned.columns = ['afspraak_afspraak_id', 'afspraak_thema', 'afspraak_subthema', 'afspraak_onderwerp', 'afspraak_eindtijd', 'afspraak_keyphrases']

In [307]:
# afspraak_betreft_account_cleaned["afspraak_betreft"] = "account"
# afspraak_account_gelinkt_cleaned["afspraak_betreft"] = "account"
# afspraak_betreft_contact_cleaned["afspraak_betreft"] = "contact"

In [308]:
afspraken_merged = pd.concat([afspraak_betreft_account_cleaned, afspraak_account_gelinkt_cleaned, afspraak_betreft_contact_cleaned])

In [309]:
afspraken = pd.read_csv('../data_clean/Afspraak_alle_fixed.csv', sep=',')
afspraken.columns = ['afspraak_afspraak_id']
afspraken.head()

,afspraak_afspraak_id
0,00082DAA-24BF-ED11-83FF-6045BD8952CE
1,00109C77-4D09-EB11-8114-001DD8B72B61
2,0011A5D9-CC9A-EC11-B400-0022488005A7
3,0016532E-14E3-EC11-BB3C-000D3ABCA38B
4,003760C1-13B4-EC11-983F-00224883C880


In [310]:
afspraken_merged = pd.merge(afspraken, afspraken_merged, on='afspraak_afspraak_id', how='inner')
afspraken_merged.reset_index(drop=True, inplace=True)

In [311]:
afspraken_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10362 entries, 0 to 10361
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   afspraak_afspraak_id  10362 non-null  object
 1   afspraak_thema        10362 non-null  object
 2   afspraak_subthema     10362 non-null  object
 3   afspraak_onderwerp    10362 non-null  object
 4   afspraak_eindtijd     10362 non-null  object
 5   afspraak_keyphrases   10362 non-null  object
dtypes: object(6)
memory usage: 485.8+ KB


In [312]:
# RAM vrijmaken
try:
    del afspraak_betreft_account_cleaned
    del afspraak_account_gelinkt_cleaned
    del afspraak_betreft_contact_cleaned
    del afspraken
except:
    print("Data is niet meer beschikbaar / is al verwijderd")

In [313]:
afspraken_merged = afspraken_merged.drop_duplicates(subset=['afspraak_afspraak_id'], keep='first')
afspraken_merged.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7624 entries, 0 to 10360
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   afspraak_afspraak_id  7624 non-null   object
 1   afspraak_thema        7624 non-null   object
 2   afspraak_subthema     7624 non-null   object
 3   afspraak_onderwerp    7624 non-null   object
 4   afspraak_eindtijd     7624 non-null   object
 5   afspraak_keyphrases   7624 non-null   object
dtypes: object(6)
memory usage: 416.9+ KB


In [314]:
activiteit_vereist_contact = pd.read_csv('../data_clean/Activiteit_vereist_contact_fixed.csv', sep=',')
activiteit_vereist_contact.shape

(4635, 2)

In [315]:
afspraken_merged = afspraken_merged.merge(activiteit_vereist_contact, left_on='afspraak_afspraak_id', right_on='activiteitvereistcontact_activityid_id', how='inner')
afspraken_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4545 entries, 0 to 4544
Data columns (total 8 columns):
 #   Column                                  Non-Null Count  Dtype 
---  ------                                  --------------  ----- 
 0   afspraak_afspraak_id                    4545 non-null   object
 1   afspraak_thema                          4545 non-null   object
 2   afspraak_subthema                       4545 non-null   object
 3   afspraak_onderwerp                      4545 non-null   object
 4   afspraak_eindtijd                       4545 non-null   object
 5   afspraak_keyphrases                     4545 non-null   object
 6   activiteitvereistcontact_activityid_id  4545 non-null   object
 7   activiteitvereistcontact_reqattendee    4545 non-null   object
dtypes: object(8)
memory usage: 284.2+ KB


In [316]:
afspraken_merged['afspraak_thema'].value_counts()

afspraak_thema
Lidmaatschap                                       3446
Welt 2.0                                            371
Plato                                               313
Internationaal Ondernemen                            74
Digitalisering, IT & Technologie                     71
Innovatie                                            56
Welt 2.0-2023                                        53
Groeien                                              50
Welt                                                 29
Duurzaam Ondernemen                                  22
Familiebedrijven                                     12
Arbeidsmarkt                                         10
Algemeen Management                                   9
Human Resources                                       8
Opvolging en Overname                                 4
Financieel                                            3
Netwerking                                            3
Mobiliteit                       

#### 3. Alles van Campagnes mergen

##### Campagne mergen met Sessie

In [317]:
campagne = pd.read_csv('../data_clean/Campagne_fixed.csv', sep=',')
campagne.shape

(4092, 11)

In [318]:
sessie = pd.read_csv('../data_clean/Sessie_fixed.csv', sep=',')
sessie.shape

(9403, 8)

In [319]:
sessie_themes_grouped = {
  'Bryo': ['Bryo'],
  'Algemeen': ['Familiebedrijven','Opvolging en Overname','Algemeen Management','Human Resources','Algemeen Management - Intern','Bestuurlijke organisaties'],
  'Onderwijs': ['Opleidingen','Persoonlijke vaardigheden','Onderwijs'],
  'Logistiek': ['Logistiek en Transport','Haven','Supply Chain','Retail'],
  'Welt': ['Welt', 'Welt 2.0', 'Welt 2.0-2023'],
  'Ondernemen': ['Starten', 'Internationaal Ondernemen', 'Jong Voka', 'Groeien', 'Stille Kampioenen', 'Samen doorgaan', 'Strategie'],
  'Duurzaamheid': ['Energie', 'Duurzaam Ondernemen', 'Milieu', 'Mobiliteit'],
  'Lidmaatschap': ['Lidmaatschap'],
  'Innovatie en Technologie': ['Innovatie', 'Digitalisering, IT & Technologie'],
  'Netwerking': ['Netwerking'],
  'Economie': ['Arbeidsmarkt', 'Economie', 'Fiscaal', 'Financieel', 'Marketing & Sales', 'Jobkanaal'],
  'Juridisch': ['Bedrijfsjuridisch', 'Juridisch'],
  'Veiligheid en communicatie': ['Communicatie', 'Veiligheid & Preventie', 'Welzijn en gezondheidszorg'],
  'Andere': ['Plato', 'Ruimtelijke ordening en Infrastructuur', 'Regeringsvorming', 'Industrie', 'Aankoop', 'Privé&Vrije tijd', 'Aantrekkelijke regio', 'Coronavirus']
}

def map_thema(thema):
    for category, themes in sessie_themes_grouped.items():
        if thema in themes:
            return category
    return thema

sessie['sessie_thema_naam_'] = sessie['sessie_thema_naam_'].apply(map_thema)

In [320]:
sessie['sessie_thema_naam_'].value_counts()

sessie_thema_naam_
Algemeen                      2914
Ondernemen                    1185
Economie                      1083
Netwerking                     993
Onderwijs                      886
Innovatie en Technologie       571
Bryo                           533
unknown                        403
Duurzaamheid                   242
Welt                           211
Veiligheid en communicatie     170
Logistiek                      108
Lidmaatschap                    41
Andere                          37
Juridisch                       26
Name: count, dtype: int64

In [321]:
result = (sessie.assign(sessie_thema_naam_list=sessie['sessie_thema_naam_'].str.split(', '))
               .explode('sessie_thema_naam_list')
               .drop_duplicates()
               .groupby('sessie_campagne')['sessie_thema_naam_list']
               .agg(lambda x: list(set(x)))
               .reset_index()
               .sort_values(by='sessie_thema_naam_list', key=lambda x: x.str.len(), ascending=False))
result

,sessie_campagne,sessie_thema_naam_list
1492,61FE6D67-F088-E811-80F3-001DD8B72B61,"[Juridisch, Innovatie en Technologie, Ondernem..."
3209,CFF2BA02-E99C-EB11-811E-001DD8B72B62,"[Innovatie en Technologie, Ondernemen, Duurzaa..."
3066,C6D72260-E451-EC11-8C62-000D3ABFCF4A,"[Juridisch, Innovatie en Technologie, Ondernem..."
2414,9D0868DB-952A-EC11-8124-001DD8B72B61,"[Juridisch, Innovatie en Technologie, Ondernem..."
976,416A235F-F209-E911-80FA-001DD8B72B62,"[Innovatie en Technologie, Ondernemen, Duurzaa..."
...,...,...
1340,5865C940-3455-E511-8718-005056B06EC4,[Onderwijs]
1341,58665379-0E28-E211-B2FD-005056B06EB4,[unknown]
1342,587BE6F8-A255-E711-80E7-001DD8B72B61,[Welt]
1343,588A47E4-6B68-E711-80E8-001DD8B72B61,[Ondernemen]


In [322]:
unique_categories = set(category for row in result['sessie_thema_naam_list'] for category in row)

for category in unique_categories:
    result[category] = result['sessie_thema_naam_list'].apply(lambda x: int(category in x))

result.drop('sessie_thema_naam_list', axis=1, inplace=True)

In [323]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3951 entries, 1492 to 3950
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   sessie_campagne             3951 non-null   object
 1   Juridisch                   3951 non-null   int64 
 2   Netwerking                  3951 non-null   int64 
 3   Lidmaatschap                3951 non-null   int64 
 4   Logistiek                   3951 non-null   int64 
 5   Innovatie en Technologie    3951 non-null   int64 
 6   Ondernemen                  3951 non-null   int64 
 7   Onderwijs                   3951 non-null   int64 
 8   Duurzaamheid                3951 non-null   int64 
 9   Andere                      3951 non-null   int64 
 10  Economie                    3951 non-null   int64 
 11  Welt                        3951 non-null   int64 
 12  Bryo                        3951 non-null   int64 
 13  Algemeen                    3951 non-null   int64 

In [324]:
campagne["campagne_type_campagne"].value_counts()

campagne_type_campagne
Netwerkevenement    1751
Opleiding           1268
Project              565
Infosessie           394
Projectgebonden       99
Campagne              15
Name: count, dtype: int64

In [325]:
campagnes_merged = campagne.merge(result, left_on='campagne_campagne_id', right_on='sessie_campagne', how='inner')
campagnes_merged.shape

(3951, 27)

#### CDI

In [326]:
accounts_merged.shape

(91783, 37)

In [327]:
df_visit = pd.read_csv('../data_clean/CDI_visits_fixed.csv', sep=',')

df_visit.drop(
    ['visit_ip_postcode', 'visit_aangemaakt_op', 'visit_adobe_reader',
    'visit_contact_naam_', 'visit_containssocialprofile', 'visit_ended_on', 'visit_ip_address',
    'visit_ip_organization', 'visit_keywords', 'visit_ip_longitude', 'visit_ip_latitude', 'visit_referrer', 
    'visit_score', 'visit_started_on', 'visit_ip_status', 'visit_visit_id', 'visit_gewijzigd_op',
    'visit_entry_page', 'visit_exit_page', 'visit_ip_company', 'visit_referring_host', 'visit_referrer_type', 
    'visit_bounce', 'visit_duration', 'visit_browser', 'visit_ip_stad', 'visit_ip_land', 'visit_operating_system',
   ], axis=1, inplace=True)

df_visit.drop_duplicates(inplace=True)

df_visit['visit_first_visit'] = df_visit['visit_first_visit'] \
      .str.replace('Ja', '1').str.replace('Nee', '0') \
      .str.replace('unknown', '-1').astype(int)

df_visit['visit_total_pages'] = df_visit['visit_total_pages']\
      .replace('unknown', '-1.0').astype(float)

##############################################################################################################

# Csv files inlezen
df_mailing = pd.read_csv('../data_clean/CDI_mailing_fixed.csv', sep=',')
df_mailing.drop(['mailing_sent_on'], axis=1, inplace=True)
df_mailing.drop_duplicates(inplace=True)
df_click = pd.read_csv('../data_clean/CDI_sent_email_clicks_fixed.csv', sep=',')
df_click.drop(['sentemail_kliks_contact', 'sentemail_kliks_sent_email_id'], axis=1, inplace=True)
df_click.drop_duplicates(inplace=True)

# Merge en drop kolommen
df_mailing_merged = df_mailing.merge(df_click, left_on='mailing_mailing_id', right_on='sentemail_kliks_e_mail_versturen', how='inner')
df_mailing_merged.drop(['sentemail_kliks_e_mail_versturen'], axis=1, inplace=True)

# Voeg een nieuwe kolom toe voor het aantal mails per groep
df_mailing_merged['aantal_mails'] = df_mailing_merged.groupby(
    ['mailing_mailing_id'])['mailing_mailing_id'].transform('count')

# Sentmail_kliks_clicks optellen per mailing
df_mailing_merged = df_mailing_merged.groupby(['mailing_mailing_id', 'mailing_name', 'mailing_onderwerp'], 
                          as_index=False).agg({'sentemail_kliks_clicks': 'sum', 
                                                **{col: 'first' for col in df_mailing_merged.columns if col not in ['sentemail_kliks_clicks']}})

##############################################################################################################

df_visit_mailing = df_visit.merge(df_mailing_merged, left_on='visit_email_send', right_on='mailing_mailing_id', how='left')
df_visit_mailing.drop(['visit_email_send', 'mailing_mailing_id', 'visit_campaign'], axis=1, inplace=True)

# String kolommen cleanen
df_visit_mailing['mailing_name'] = df_visit_mailing['mailing_name'].str.replace('OV-', '') \
                                            .str.replace('OV ', '').str.replace('OV -', '') \
                                            .str.replace('-', ' ').str.replace(r'[^\w\s]', ' ', regex=True) \
                                            .str.replace('  ', ' ').str.strip().str.lower().str.replace('  ', ' ')

df_visit_mailing['mailing_onderwerp'] = df_visit_mailing['mailing_onderwerp'] \
                                            .str.replace('-', ' ').str.replace(r'[^\w\s]', ' ', regex=True) \
                                            .str.replace('  ', ' ').str.strip().str.lower().str.replace('  ', ' ')

df_visit_mailing['sentemail_kliks_clicks'] \
    = df_visit_mailing['sentemail_kliks_clicks'].replace('unknown', '0').astype(int)

In [328]:
accounts_merged = accounts_merged.merge(df_visit_mailing, left_on='contact_contactpersoon_id', right_on='visit_contact', how='left')
# Numeric en categorical kolommen selecteren
num_cols = accounts_merged.select_dtypes(include=['float64', 'int64']).columns
cat_cols = accounts_merged.select_dtypes(include=['object']).columns

# fill NaN values with -1 for numeric columns
accounts_merged[num_cols] = accounts_merged[num_cols].fillna(-1)
# fill NaN values with 'unknown' for categorical columns
accounts_merged[cat_cols] = accounts_merged[cat_cols].fillna('unknown')

# verander alle int64 en int32 naar int8
int_cols = accounts_merged.select_dtypes(include=['int64', 'int32']).columns
accounts_merged[int_cols] = accounts_merged[int_cols].astype('int8')

In [329]:
# mail_click_freq berekenen
accounts_merged['mail_click_freq'] = accounts_merged.apply(
    lambda row: 0 if row['aantal_mails'] == -1 else round(row['sentemail_kliks_clicks'] / row['aantal_mails']), axis=1)
accounts_merged.drop(['sentemail_kliks_clicks', 'aantal_mails'], axis=1, inplace=True)

# marketing_pressure berekenen
accounts_merged['marketing_pressure'] = accounts_merged['marketing_pressure'] + accounts_merged['mail_click_freq'] 
+ accounts_merged['visit_total_pages'] + accounts_merged['visit_first_visit']

accounts_merged.drop(['mail_click_freq', 'visit_total_pages', 'visit_first_visit', 'visit_campagne_code', 'visit_contact', 'visit_time', 'mailing_name', 'mailing_onderwerp'], axis=1, inplace=True)
accounts_merged.shape

(115974, 37)

In [330]:
accounts_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115974 entries, 0 to 115973
Data columns (total 37 columns):
 #   Column                                                     Non-Null Count   Dtype  
---  ------                                                     --------------   -----  
 0   account_account_id                                         115974 non-null  object 
 1   account_adres_geografische_regio                           115974 non-null  object 
 2   account_adres_geografische_subregio                        115974 non-null  object 
 3   account_adres_plaats                                       115974 non-null  object 
 4   account_adres_postcode                                     115974 non-null  object 
 5   account_adres_provincie                                    115974 non-null  object 
 6   account_industriezone_naam_                                115974 non-null  object 
 7   account_is_voka_entiteit                                   115974 non-null  int8   

#### 4. Accounts mergen met Afspraken

In [331]:
onnodige_kolommen_accounts = [
 'account_account_id',
 'account_adres_postcode',
 'account_industriezone_naam_',
 'account_oprichtingsdatum',
 'account_adres_plaats',
 'account_voka_nr_',
 'activiteitscode_naam',
 'account_is_voka_entiteit',
 'account_status',
 'account_adres_geografische_regio',
 'account_adres_provincie',
 'account_reden_van_status',
 'account_adres_land',
 'account_activiteitscode_account',
 'account_activiteitscode_activiteitscode',
 'account_activiteitscode_inf_account_inf_activiteitscodeid',
 'activiteitscode_activiteitscode_id',
 'contact_account',
 'contact_functietitel',
 'contact_persoon_id',
 'contact_voka_medewerker',
 'contact_status',
 'contactfunctie_contactpersoon',
 'contactfunctie_functie',
 'functie_functie_id',
 'activiteitscode_status',
 'persoon_persoonsnr_',
 'persoon_web_login',
 'persoon_reden_van_status',
 'persoon_persoon_id',
]

accounts_merged = accounts_merged.drop(onnodige_kolommen_accounts, axis=1)

In [332]:
grouping_categories = {
    'Lidmaatschap': ['Lidmaatschap'],
    'Welt': ['Welt', 'Welt 2.0', 'Welt 2.0-2023'],
    'Plato': ['Plato'],
    'Internationaal Ondernemen': ['Internationaal Ondernemen', 'Internationaal Ondernemen - voor Info en Advies'],
    'Technologie en Innovatie': ['Digitalisering, IT & Technologie', 'Innovatie'],
    'Groeien en Netwerking': ['Groeien', 'Netwerking'],
    'Duurzaamheid': ['Duurzaam Ondernemen', 'Mobiliteit'],
    'Familiebedrijfsmanagement': ['Familiebedrijven', 'Opvolging en Overname'],
    'Arbeidsmarkt': ['Arbeidsmarkt', 'Opleidingen'],
    'Bedrijfsbeheer': ['Algemeen Management', 'Bestuurlijke organisaties', 'Human Resources'],
    'Financieel': ['Financieel', 'Marketing & Sales'],
    'Logistiek en Transport': ['Logistiek en Transport', 'Haven']
}

def map_thema(thema):
    for category, themes in grouping_categories.items():
        if thema in themes:
            return category
    return thema

afspraken_merged['afspraak_thema_grouped'] = afspraken_merged['afspraak_thema'].apply(map_thema)
afspraken_merged = afspraken_merged.drop(['afspraak_thema', 'afspraak_subthema'], axis=1)

In [333]:
afspraken_merged = afspraken_merged.drop(['afspraak_afspraak_id', 'afspraak_onderwerp', 'afspraak_eindtijd', 'activiteitvereistcontact_activityid_id', 'afspraak_keyphrases'], axis=1)
afspraken_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4545 entries, 0 to 4544
Data columns (total 2 columns):
 #   Column                                Non-Null Count  Dtype 
---  ------                                --------------  ----- 
 0   activiteitvereistcontact_reqattendee  4545 non-null   object
 1   afspraak_thema_grouped                4545 non-null   object
dtypes: object(2)
memory usage: 71.1+ KB


In [334]:
afspraakthema_counts = afspraken_merged.groupby('activiteitvereistcontact_reqattendee')['afspraak_thema_grouped'].value_counts().unstack(fill_value=0)
accounts_merged = accounts_merged.merge(afspraakthema_counts, left_on='contact_contactpersoon_id', right_index=True, how='left').fillna(0)
accounts_merged.head()

,account_adres_geografische_subregio,account_ondernemingsaard,account_ondernemingstype,account_primaire_activiteit,contact_contactpersoon_id,functie_naam,marketing_pressure,Arbeidsmarkt,Bedrijfsbeheer,Duurzaamheid,Familiebedrijfsmanagement,Financieel,Groeien en Netwerking,Internationaal Ondernemen,Lidmaatschap,Logistiek en Transport,Plato,Technologie en Innovatie,Welt
0,Oudenaarde,Diensten,Bedrijf,Overige industrie & diensten,0A9D6946-EB72-E111-B43A-00505680000A,Contact Lidmaatschap,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Gent,Diensten,Familiebedrijf,Vastgoed,FF015050-8D55-EB11-8117-001DD8B72B61,Contact Lidmaatschap,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Gent,Diensten,Familiebedrijf,Vastgoed,FF015050-8D55-EB11-8117-001DD8B72B61,Contact Lidmaatschap,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Gent,Diensten,Familiebedrijf,Vastgoed,FF015050-8D55-EB11-8117-001DD8B72B61,Contact Lidmaatschap,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Oudenaarde,Diensten,Bedrijf,Bouw,FF0A3AF6-0D42-E611-80D6-005056B06EC4,Contact Lidmaatschap,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [335]:
# RAM vrijmaken
try:
    del afspraken_merged
    del activiteit_vereist_contact
    del result
except:
    print("Data is niet meer beschikbaar / is al verwijderd")

#### 5. Cleaning voor de grote merge

In [336]:
onnodige_kolommen_campagnes = [
 'campagne_campagne_nr',
 'campagne_einddatum',
 'campagne_naam',
 'campagne_reden_van_status',
 'campagne_naam_in_email',
 'campagne_startdatum',
 'campagne_url_voka_be',
 'campagne_status',
 'sessie_campagne',
]

campagnes_merged = campagnes_merged.drop(onnodige_kolommen_campagnes, axis=1)

In [337]:
# Drop duplicates
accounts_merged = accounts_merged.drop_duplicates(keep='first')
campagnes_merged = campagnes_merged.drop_duplicates(keep='first')

In [338]:
accounts_merged.info()

<class 'pandas.core.frame.DataFrame'>
Index: 104564 entries, 0 to 115973
Data columns (total 19 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   account_adres_geografische_subregio  104564 non-null  object 
 1   account_ondernemingsaard             104564 non-null  object 
 2   account_ondernemingstype             104564 non-null  object 
 3   account_primaire_activiteit          104564 non-null  object 
 4   contact_contactpersoon_id            104564 non-null  object 
 5   functie_naam                         104564 non-null  object 
 6   marketing_pressure                   104564 non-null  int64  
 7   Arbeidsmarkt                         104564 non-null  float64
 8   Bedrijfsbeheer                       104564 non-null  float64
 9   Duurzaamheid                         104564 non-null  float64
 10  Familiebedrijfsmanagement            104564 non-null  float64
 11  Financieel        

In [339]:
campagnes_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3951 entries, 0 to 3950
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   campagne_campagne_id        3951 non-null   object
 1   campagne_type_campagne      3951 non-null   object
 2   campagne_soort_campagne     3951 non-null   object
 3   Juridisch                   3951 non-null   int64 
 4   Netwerking                  3951 non-null   int64 
 5   Lidmaatschap                3951 non-null   int64 
 6   Logistiek                   3951 non-null   int64 
 7   Innovatie en Technologie    3951 non-null   int64 
 8   Ondernemen                  3951 non-null   int64 
 9   Onderwijs                   3951 non-null   int64 
 10  Duurzaamheid                3951 non-null   int64 
 11  Andere                      3951 non-null   int64 
 12  Economie                    3951 non-null   int64 
 13  Welt                        3951 non-null   int6

#### 6. Alles mergen

In [340]:
import pandas as pd
inschrijving = pd.read_csv('../data_clean/Inschrijving_fixed.csv', sep=',')
inschrijving.shape

(49175, 8)

In [341]:
pq.write_table(pa.Table.from_pandas(accounts_merged), 'account_merged.parquet')
pq.write_table(pa.Table.from_pandas(campagnes_merged), 'campagnes_merged.parquet')

# Read DataFrames back from Parquet files
account_merged_parquet = pd.read_parquet('account_merged.parquet')
campagnes_merged_parquet = pd.read_parquet('campagnes_merged.parquet')

In [ ]:
merged_total = pd.merge(account_merged_parquet.assign(key=1), campagnes_merged_parquet.assign(key=1), on='key').drop('key', axis=1)
merged_total.shape

In [ ]:
# RAM vrijmaken
try:
    del accounts_merged
    del campagnes_merged
    del account_merged_parquet
    del campagnes_merged_parquet
except:
    print("Data is niet meer beschikbaar / is al verwijderd")

In [ ]:
merged_total["ingeschreven"] = 0

for index, row in merged_total.iterrows():
    if ((inschrijving['inschrijving_contactfiche'] == row['contact_contactpersoon_id']) & (inschrijving['inschrijving_campagne'] == row['campagne_campagne_id'])).any():
        merged_total.at[index, 'ingeschreven'] = 1

ingeschreven
0    19754
1        1
Name: count, dtype: int64

In [ ]:
merged_total = merged_total.drop(['contact_contactpersoon_id', 'campagne_campagne_id'], axis=1)

#### 7. Alle samples nemen met inschrijving == 1 en 200k samples nemen met inschrijving == 0

In [ ]:
df_ingeschreven = merged_total[merged_total['ingeschreven'] == 1]
df_niet_ingeschreven = merged_total[merged_total['ingeschreven'] == 0].sample(n=200000, random_state=42)

merged_total = pd.concat([df_ingeschreven, df_niet_ingeschreven])
merged_total['ingeschreven'].value_counts()

#### 8. Verdere data cleaning en One/Multi hot encoding

In [ ]:
from sklearn.preprocessing import OneHotEncoder
import category_encoders as ce

oneHot = OneHotEncoder(sparse=False)

In [ ]:
# Subregio
subregio_1hot = oneHot.fit_transform(merged_total[['account_adres_geografische_subregio']])

C:\Users\marlo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
# Ondernemingstype
be_ondernemingstype = ce.BinaryEncoder(cols=['account_ondernemingstype'])
encoded_ondernemingstype = be_ondernemingstype.fit_transform(merged_total['account_ondernemingstype'])
merged_total = merged_total.join(encoded_ondernemingstype, rsuffix='_ondernemingstype')

In [ ]:
# Primaire activiteit
be_primaire_activiteit = ce.BinaryEncoder(cols=['account_primaire_activiteit'])
encoded_primaire_activiteit = be_primaire_activiteit.fit_transform(merged_total['account_primaire_activiteit'])
merged_total = merged_total.join(encoded_primaire_activiteit, rsuffix='_primaire_activiteit')

In [ ]:
# Ondernemingsaard
diensten_column = []
productie_column = []

for label in merged_total["account_ondernemingsaard"]:
    if label == "Diensten":
        diensten_column.append(1)
        productie_column.append(0)
    elif label == "Productie":
        diensten_column.append(0)
        productie_column.append(1)
    elif label == "Productie & Diensten":
        diensten_column.append(1)
        productie_column.append(1)
    elif label == "unknown":
        diensten_column.append(0)
        productie_column.append(0)

ondernemingsaard_multihot = pd.DataFrame({"Diensten": diensten_column, "Productie": productie_column})
merged_total = merged_total.join(ondernemingsaard_multihot, rsuffix='_ondernemingsaard')

In [ ]:
# Functie_naam
def transform_role(role):
    if "Verantwoordelijke" in role:
        return "Verantwoordelijke"
    elif "Medewerker" in role:
        return "Medewerker"
    else:
        return role

merged_total['functie_naam'] = merged_total['functie_naam'].apply(transform_role)
functie_1hot = oneHot.fit_transform(merged_total[['functie_naam']])

C:\Users\marlo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
# Campagne soort
online_column = []
offline_column = []

for label in merged_total["campagne_soort_campagne"]:
    if label == "Offline":
        online_column.append(0)
        offline_column.append(1)
    elif label == "Online":
        online_column.append(1)
        offline_column.append(0)
    elif label == "On en Offline":
        online_column.append(1)
        offline_column.append(1)
    else:
        online_column.append(0)
        offline_column.append(0)

campagne_soort_multihot = pd.DataFrame({"Online": online_column, "Offline": offline_column})
merged_total = merged_total.join(campagne_soort_multihot, rsuffix='_campagne_soort')

In [ ]:
# Campagne type
campagne_type_1hot = oneHot.fit_transform(merged_total[['campagne_type_campagne']])

C:\Users\marlo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
merged_total = merged_total.drop(['account_adres_geografische_subregio', 'account_ondernemingstype', 'account_primaire_activiteit', 'account_ondernemingsaard', 'functie_naam'], axis=1)
merged_total = merged_total.drop(['campagne_soort_campagne', 'campagne_type_campagne'], axis=1)

In [ ]:
# Al de one hot encodings samenvoegen
merged_total = merged_total.join(pd.DataFrame(subregio_1hot), rsuffix='_subregio')
merged_total = merged_total.join(pd.DataFrame(functie_1hot), rsuffix='_functie')
merged_total = merged_total.join(pd.DataFrame(campagne_type_1hot), rsuffix='_campagne_type')

In [ ]:
# convert all columns to int8
merged_total = merged_total.astype('int8', errors='ignore')
merged_total = merged_total.astype('int8', errors='ignore')

In [ ]:
merged_total = merged_total.fillna(0)
merged_total = merged_total.fillna(0)

In [ ]:
# RAM vrijmaken
try:
    del ondernemingsaard_multihot
    del functie_1hot
    del campagne_soort_multihot
    del campagne_type_1hot
except:
    print("Data is niet meer beschikbaar / is al verwijderd")


#### 9. Dataset opslaan

In [ ]:
# Save to csv
merged_total.to_csv('../data_clean/AA_Supervised_dataset.csv', index=False)